In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
# parameter settings
is_plot = False
start=0
stride = 12
end=120

# Data Loading

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object('../data/landmarks/6kmh_softbra_8markers_12fps.pkl')
landmarks.interp_field()

In [ ]:
from mesh4d import obj3d
from mesh4d.analysis import reg_rbf

o3_ls = obj3d.load_obj_series(
    folder='../data/meshes/6kmh_softbra_8markers/',
    start=start,
    stride=stride,
    end=end,
    obj_type=reg_rbf.Obj3d_RBF,
)

# Displacement Field Estimation

In [ ]:
o4 = reg_rbf.Obj4d_RBF(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=True,
)
o4.add_obj(*o3_ls, landmarks=landmarks)
o4.load_markerset('M8', landmarks)

In [ ]:
import pyvista as pv
pv.set_jupyter_backend('static')

if is_plot:
    o3_ls[0].show()

In [ ]:
if is_plot:
    o3_ls[0].show_deform(mode='nonrigid')

In [ ]:
if is_plot:
    o3_ls[0].show_deform(mode='rigid')

In [ ]:
obj_deform = o3_ls[0].get_deform_obj3d()
obj_gt = o3_ls[1]

if is_plot:
    obj3d.Obj3d_Kps.show_diff(obj_deform, obj_gt)

# Ultra-dense Motion Capture

In [ ]:
kps_source = landmarks.get_time_coord(0)
o4.vkps_track(kps_source, frame_id=0)

In [ ]:
if is_plot:
    o4.show_deform_gif(output_folder='output', filename='obj4d_vkps_rbf', kps_names=('vkps', 'M8'))

Assemble the tracked virtual key points

In [ ]:
vkps = o4.assemble_markerset(name='vkps')

In [ ]:
if is_plot:
    vkps.plot_frame(frame_id=0)

In [ ]:
from mesh4d import kps
diff = kps.MarkerSet.diff(vkps, landmarks)